**122: 1. Veri seti, Multi-Output Random Forest + GridSearchCV Modelleme (Sadece X1 - Price Lag Rolling Mean)**  
TRAIN

Amaç: 16 ürün için çok çıkışlı regresyon tahmini yapmak.

Girdi:  
- X1_train_multi.npy → Fiyat gecikme rolling mean özellikleri

Çıktı: Her ürün için satış tahminleri.

Model: Random Forest Regressor modeli, Multi-Output yaklaşımı (sklearn MultiOutputRegressor)

Yöntem: GridSearchCV ile hiperparametre optimizasyonu (n_estimators, max_depth, min_samples_split, min_samples_leaf)

Metrik: Mean Squared Error (MSE)

Çıktılar:  
- GridSearch CV sonuçları raporlanır.  
- En iyi model: `models/122_rf_X1_model.pkl` dosyasına kaydedilir.


1 - KÜTÜPHANE & VERİ YÜKLEME

In [4]:
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import joblib

X_train = np.load('npy/X1_train_multi.npy')
y_train = np.load('npy/y_train_multi.npy')

print(f"X1_train shape: {X_train.shape} | y_train shape: {y_train.shape}")

X1_train shape: (210239, 48) | y_train shape: (210239, 16)


2 - MODEL & GRIDSEARCHCV

In [5]:
rf = RandomForestRegressor(random_state=42)
multi_rf = MultiOutputRegressor(rf)

param_grid = {
    'estimator__n_estimators': [15],
    'estimator__max_depth': [3],
    'estimator__min_samples_split': [2],
    'estimator__min_samples_leaf': [2],
}

scorer = make_scorer(mean_squared_error, greater_is_better=False)

grid_search = GridSearchCV(
    estimator=multi_rf,
    param_grid=param_grid,
    scoring=scorer,
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("\nEn iyi parametre kombinasyonu:")
print(grid_search.best_params_)
print(f"En iyi CV MSE: {grid_search.best_score_:.4f}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits

En iyi parametre kombinasyonu:
{'estimator__max_depth': 3, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 15}
En iyi CV MSE: -0.2032


3 - MODEL KAYIT

In [6]:
best_model = grid_search.best_estimator_
joblib.dump(best_model, 'models/122_rf_X1_model.pkl')
print("En iyi model 'models/122_rf_X1_model.pkl' dosyasına kaydedildi.")

En iyi model 'models/122_rf_X1_model.pkl' dosyasına kaydedildi.
